In [11]:
from dotenv import load_dotenv
from langchain_classic import hub
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents.react.agent import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

load_dotenv()

True

In [12]:
react_prompt = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question should be in the format {format_instructions}

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [13]:
from typing import List
from pydantic import Field, BaseModel

class Source(BaseModel):
    """Schema for a source that the agent uses"""
    url: str = Field(description="The URL of the source used by an agent")

class AgentResponse(BaseModel):
    """Schema for agent's response to the query"""
    answer: str = Field(description="Agent's answer to the given query")
    sources: List[Source] = Field(default_factory=list, description="list of sources used by the agent")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [TavilySearch()]
tool_names = [tool.name for tool in tools]
output_parser = PydanticOutputParser(pydantic_object=AgentResponse)

prompt = PromptTemplate.from_template(react_prompt)
prompt.partial_variables = {"tools": tools, "tool_names":tool_names, "format_instructions": output_parser.get_format_instructions()}
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
print(type(agent))
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
print(type(agent_executor))
chain = agent_executor

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_classic.agents.agent.AgentExecutor'>


In [15]:
result = chain.invoke(input={"input": "search for 3 job posting for an ai engineer in bengaluru area and list their details"})
print(result)



> Entering new AgentExecutor chain...
I need to use the search engine to find information about current job postings for an AI engineer position in the Bengaluru area. However, I must ensure to gather details for at least three different job postings. Where available, I should gather details on the company name, job posting URL, job title, location, and job description.
Action: tavily_search
Action Input: "AI engineer job postings in Bengaluru"{'query': 'AI engineer job postings in Bengaluru', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.glassdoor.com/Job/bengaluru-ai-and-ml-engineer-jobs-SRCH_IL.0,9_IC2940587_KO10,28.htm', 'title': '2056 ai and ml engineer Jobs in Bengaluru, November 2025', 'content': '+ software engineer jobs in KarnatakaKarnataka jobseBay jobs in Karnataka. + machine learning engineer jobs in KarnatakaKarnataka jobseBay jobs in Karnataka. + data engineer jobs in KarnatakaKarnataka jobsClarivate jobs in Karnataka. + so